In [34]:
from itertools import combinations

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import combinations

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import warnings

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from unidecode import unidecode
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from itertools import combinations
from datetime import datetime
from contextlib import contextmanager

In [35]:
df_train = pd.read_csv("../../data/kalapa/train.csv")
df_test = pd.read_csv("../../data/kalapa/test.csv")

/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
df_train.Field_3

0        1.0
1        NaN
2        2.0
3        NaN
4        NaN
        ... 
53025    1.0
53026    NaN
53027    2.0
53028    NaN
53029    NaN
Name: Field_3, Length: 53030, dtype: float64

In [37]:
def check_duplicate(feature1, feature2):
    check1 = list(df_train[feature1].fillna("NaN"))
    check2 = list(df_train[feature2].fillna("NaN"))
    return check1 == check2
distinct_columns = {}
for col1 in df_test.columns:
    vc = df_train[col1].value_counts()
    index = [len(vc), set(list(vc)), set(list(vc.index))]
    for col2 in distinct_columns:
        if index == distinct_columns[col2] and check_duplicate(col1, col2):
            break
    else:
        distinct_columns[col1] = index
duplicate = []
for col in df_train.columns:
    if col not in distinct_columns:
        duplicate.append(col)

In [38]:
distinct_columns.keys()

dict_keys(['id', 'Field_1', 'Field_2', 'Field_3', 'Field_4', 'Field_5', 'Field_6', 'Field_7', 'Field_8', 'Field_9', 'Field_10', 'Field_11', 'Field_12', 'Field_13', 'Field_15', 'Field_18', 'Field_19', 'Field_20', 'Field_21', 'Field_22', 'Field_23', 'Field_25', 'Field_27', 'Field_28', 'Field_29', 'Field_32', 'Field_33', 'Field_34', 'Field_35', 'ngaySinh', 'namSinh', 'gioiTinh', 'diaChi', 'Field_36', 'Field_38', 'Field_39', 'Field_40', 'Field_41', 'Field_42', 'Field_43', 'Field_44', 'Field_45', 'Field_46', 'Field_47', 'Field_48', 'Field_49', 'Field_50', 'Field_51', 'Field_53', 'Field_54', 'Field_55', 'Field_56', 'Field_58', 'Field_59', 'Field_60', 'Field_61', 'Field_62', 'Field_63', 'Field_64', 'Field_65', 'Field_66', 'Field_67', 'Field_68', 'maCv', 'Field_69', 'Field_70', 'Field_71', 'Field_72', 'Field_73', 'Field_74', 'Field_75', 'Field_76', 'Field_77', 'friendCount', 'info_social_sex', 'subscriberCount', 'currentLocationLocationId', 'currentLocationLatitude', 'currentLocationLongitude'

In [39]:
DROP = ["gioiTinh","info_social_sex", "diaChi", 'currentLocationCity', 'currentLocationName',\
        'homeTownCity', 'homeTownName', "Field_68", "Field_49", "Field_48", "Field_46", "Field_45", "Field_45", \
       "Field_56", "Field_61"]
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 34, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [40]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [41]:
def subtract_date(date1,date2, df):
    df[date1] = pd.to_datetime(df[date1], infer_datetime_format=True)
    df[date2] = pd.to_datetime(df[date2], infer_datetime_format=True)
    df[date1+date2] = (df[date2] - df[date1]).dt.days

In [42]:
def process_ngaySinh(s):
    if s != s:
        return np.nan
    try:
        s = int(s)
    except ValueError:
        s = s.split(" ")[0]
        
    return datetime.strptime(str(s)[:6], "%Y%m")

In [43]:
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj

In [44]:
def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj

In [45]:
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        if col == "Field_34":
            continue
        df[col] = df[col].apply(date_normalize)

    df["Field_34"] = df["Field_34"].apply(process_ngaySinh)
    df["ngaySinh"] = df["ngaySinh"].apply(process_ngaySinh)
    
    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%d-%m-%Y')
    
    subtract_date('Field_5','Field_6',df)
    subtrac_List = ['Field_1', 'Field_2', 'Field_43', 'Field_44', 'Field_7','Field_8', 'Field_9']
    subtract_2C = list(combinations(subtrac_List, 2))
    for l in subtract_2C:
        subtract_date(l[0],l[1],df)
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        subtract_date(f'{cat}_startDate', f'{cat}_endDate', df)
    print(df.shape) 
    return df

In [46]:
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s

In [47]:
def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)
    return df

In [48]:
def combine_gender(s):
    x, y = s
    return x if x != None else y if y != None else None

In [49]:
def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1).astype("category")
    return df

In [50]:
def process_ordinal(df):        
    df["subscriberCount"].replace(0, np.nan, inplace=True)
    df["friendCount"].replace(0, np.nan, inplace=True)
    
    df["Field_13"] = df["Field_13"].apply(lambda x: 1 if x == x else 0)
    df["Field_38"] = df["Field_38"].map({0: 0.0, 1: 1.0, "DN": np.nan, "TN": np.nan, "GD": np.nan})
    df["Field_62"] = df["Field_62"].map({"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "Ngoài quốc doanh Quận 7": np.nan})
    df["Field_47"] = df["Field_47"].map({"Zezo": 0, "One": 1, "Two": 2, "Three": 3, "Four": 4})
    
    df["Field_27"] = df["Field_27"].replace({0.0: np.nan})
    df["Field_28"] = df["Field_28"].replace({0.0: np.nan})
        
    for col in df.columns:
        if df[col].dtype.name == "object":
            df[col] = df[col].apply(str_normalize).astype("category")
            
    return df

In [51]:
def fix_maCv(x):
    lower = str(x).lower()
    if any([x in lower for x in ["nhân viên", "kế toán", "thủ kho", "tạp vụ",\
                                "thu ngân", "phóng viên", "giao dịch", "operator", "cnv",\
                                "huấn luyện viên", "ktoán", "nv", "lễ tân", "thư ký ",\
                                "gsbh", "tiếp viên", "nhõn viờn", "chăm sóc", "hướng dẫn viên", \
                                "kiểm phẩm",  "kĩ sư",  "cấp dưỡng", "kiểm gấp",\
                                "kiểm soát viên", "kiểm ngân", "nv", "lưu hóa", "nhân  viên", "lưu trữ viên", \
                                "phát hành báo", "ktv", "kiến trúc sư", "kinh tế viên", "staff", "checker",\
                                "sỹ quan", "dịch vụ", "kiểm tra viên", \
                                "cộng tác viên", "qc ksqt", "nhân vên", "kế hoạch sản xuất", "nhaõn vieõn", \
                                "thí nghiệm", "thông kê", "kiểm hàng", "k? s?", "thư ký", "hợp đồng", \
                                "thẩm phán", "thao tác viên", "thủ quỹ", "lấy mẫu", "LÁI XE".lower(), \
                                "biên tập", "ksxd", "phiên dịch", "tb tuyên huấn", "sales", "thường trực", \
                                "trợ lý", "tiếp thực", "giỏm sỏt", \
                                "trình dược viên", "btđtn", "phúng viờn", \
                                "thông dịch", "tiếp thị", "tư vấn", "tổng đài", "vi tính", "viên chức", \
                                "văn phòng", "văn thư", 'Thiết Kế'.lower(), "vận hành", "nhan vien", \
                                "kiểm tra chất lượng", "pbt", "phi công", "laptop",\
                                "phụ trách kiểm hàng", "quay phim", "thư viện viên", \
                                "trực buồng", "techcombank", "cảng vụ viên", "trainee", "bhxh", "marketing", \
                                "tuyên truyền viên",  "kế hoạch", "ks", \
                                "an toàn viên", "CHỈ SỐ VIÊN".lower(), \
                                "thu mua", "giám éịnh viên", "kiểm định viên",  "giám định", \
                                "quản trị mạng", "thơợ", "thống kê", "thủ  quỹ",  \
                                "điện thoại", "điện thọai", "đội thông tin", "đón  tiếp khách hàng", \
                                "đơn hàng", "đại diện", "điều phối viên", "n.v", "bán vé", "c«ng nh©n", \
                                "gs camera",  "máy dán", "tv",  "đồ họa"]]):
        return "NV"
    elif any([x in lower for x in ["kỹ","chuyên viên","kĩ thuật viên","kĩ thuật", "kỹ thuật viên",\
                                 "kỷ thuật viên","lập trình viên", "chuyờn viờn", "chuyªn viªn",]]):
        return "KTV"
    elif any([x in lower for x in ["cán bộ", "cán sự", "bí thư đoàn", "cb", "phường đội", "trưởng công an", \
                                  "công an viên", "an ninh viên","tổ viên", "công chức", "quản sinh", \
                                   "bí thư xã  đoàn","điều tra viên", "đăng kiểm", "cỏn bộ","đtv", "trực ban",\
                                  ]]):
        return "CB"
    elif any([x in lower for x in ["tổ trưởng", "giám sát", "quản lý", "cửa hàng trưởng", "chuyền trưởng", "kho",\
                                  "phó", "pp", "nhóm trưởng", "trưởng phòng", "trưởng ca", "trưởng tàu khách", \
                                  "trưởng nhóm", "cửa hàng trưởng", "trưởng bộ", "tổng phụ trách", "đào tạo", \
                                  "phụ trách phòng", "quản doanh", "vhtt", "trưởng ban", "dự án", "nhân sự",\
                                  "chỉ huy", "triển khai", "trưởng kiểm soát", "điều hành", "đội trưởng", \
                                  "chỉ huy", "qlsx", "t? tr??ng", "may"]]):
        return "QL"
    elif any([x in lower for x in ["công",  "bảo vệ", "thợ", "lao động", "bán hàng",\
                                "phụ kho", "phụ việc", "phết", "bảo mẫu", "phục vụ", "đóng gói", "ép cao tần", \
                                "cn", "sản Phẩm", "giao nhận","cụng nhõn", "gia công", "kiÓm tra".lower(), \
                                "cô nuôi", "bảo dưỡng", "c.n", "coi bồn", "cạo mủ", "coõng nhaõn", "khai thác",\
                                "cod", "bốc xếp", "bếp", "lái máy", "lđpt", "khuôn", "hàng rong", "phụ bếp",\
                                "phụ tá", "thủ công", "dệt", "sửa chữa", "băng chuyền", "lắp ráp", \
                                "chà nhám", "cn", "bếp phụ", "chụp hình", "KHUNG SắT".lower(), "kcs chuyền", \
                                "lao động", "thành hình", "lao công", "cônh nhân", "thi công", "cán lần 1", \
                                "lái phụ máy", "thủy thủ", "gác ghi", "quay ghi", "bảo trì", "bảo mẩu", \
                                "máy 3 tàu", "giao hàng", "trực chính", "hỗ trợ", "lái xúc gạt", "công  nhân",\
                                "lđhđ", "lao d?ng", "c. nhân", "pv nước", "hđlđ", "vệ sinh", "phụ", "vệ sỹ", \
                                "xếp hộp", "barista", "hàn điện", "thuỷ thủ", "xây dựng", "b?o v?", \
                                "mài phá", "xếp da", "worker", "cong nhan", "pha chế", \
                                "máy trưởng", "phụ trách sản xuất", "trải vải", "YếN TIệC".lower(), "gấp xếp", \
                                "in sơ đồ", "lđ phổ thông", "mài", "nhồi mút", "ép", "gói", "đóng", \
                                "đội viên", "đánh máy", "đứng máy", "ủi", "săm", 'atlđ']]):
        return "CN"
    elif any([x in lower for x in ["lái xe", "tài xế", "phụ xe","lái cẩu", "vân hành", ]]):
        return "TX"
    elif any([x in lower for x in ["giáo viên", "gv", "giảng viên", "chủ nhiệm lớp", "giáo viến", \
                                   "Giáo viến".lower(), "mầm non"]]):
        return "GV"
    elif any([x in lower for x in ["kinh doanh", "kinh  doanh"]]):
        return "KD"

    elif any([x in lower for x in ["hiệu trưởng", "chủ tịch", "pgđ", "giám đốc"]]):
        return "GĐ"
    elif any([x in lower for x in ["điều dưỡng", "hộ lý", "nữ hộ sinh", "h? lý", "hộ sinh"]]):
        return "ĐD"
    elif any([x in lower for x in ["y sĩ", "dược sỹ", "bác sĩ", "bác sỹ", "dược", "dược tá", "thú y", "y sỹ",\
                                  "y tế", "y công", "y tá", "dựơc tá", "ytth"]]):
        return "BS"
    elif any([x in lower for x in ["cử nhân", "cđ", "học viên", "tc", "cao đẳng", "cao đẵng", "trung cấp"]]):
        return "HS"
    elif any([x in lower for x in ["diễn viên", "ca sĩ"]]):
        return "DV-CS"
    elif any([x in lower for x in ["undefined", "nguyễn thế hưng", "ln091", "thieu chuc danh", "th noọng hẹt", \
                                  "875 lk", "-", "p.xnk", "cỏn b?", '22568', "unt", "01019882", "xtnt", "ltv", \
                                  "plđg hc", "mia", "pqtd", "kttntl", "llqs", "kcsmhm", "nt", "dtm", "i", "qc",\
                                  '/  /', '1', '14', 'a', 'chủ hộ', 'coõng vuù', 'kcs', 'pk b2c',\
                                   'xt2', 'kcs', 'sxct']]):
        return np.nan
    return x

In [52]:
def fix_55(x):
    if x in ['BC', 'BG', 'CN', 'CO', 'DB', 'EL', 'FO', 'H5', 'KX', 'N7', 'NR', 'SD', 'SJ', 'ST', 'WD']:
        return np.nan
    return x

In [53]:
def transform(df):
    df = process_datetime_cols(df)
    df = process_gender(df)
    df = process_location(df)
    df = process_ordinal(df)
    return df.drop(DROP, 1)

In [64]:
df_train = pd.read_csv("../../data/kalapa/train.csv")
df_test = pd.read_csv("../../data/kalapa/test.csv")
df_all = df_train.drop(['label'], 1).append(df_test)
df_all = df_all[distinct_columns]

df_all.maCv = df_all.maCv.apply(fix_maCv)
df_all.Field_55 = df_all.Field_55.apply(fix_55)



with timer("Preprocess"):
    df_all_fe = transform(df_all.copy())
    df_all_fe['Age'] = df_all_fe.ngaySinh.apply(lambda x: 2020 - x.year)
    df_all_fe = df_all_fe.drop('ngaySinh', axis = 1)
    cols_select = [x for x in df_all_fe.columns]
    df_fe = df_all_fe[cols_select]
    for col in df_fe.columns:
        if df_fe[col].dtype.name == "category" or  df_fe[col].dtype.name == "object":
            if df_fe[col].isnull().sum() > 0:
                df_fe[col] = df_fe[col].cat.add_categories(f'missing_{col}')
                df_fe[col].fillna(f'missing_{col}', inplace=True)
        else:
            df_fe[col].fillna(-9999999, inplace=True)

    df_fe.replace([np.inf, -np.inf], -9999999, inplace=True)


    y_label = df_train["label"]
    train_fe = df_fe[df_fe["id"] < df_train.shape[0]]
    test_fe = df_fe[df_fe["id"] >= df_train.shape[0]]

/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/parker/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(73411, 175)
Preprocess - done in 23s


In [60]:
df_train = pd.concat([df_train.id,df_train.label, train_fe.iloc[:,1:]], axis = 1)

In [63]:
for col in train_fe.iloc[:,2:].columns:
    vc1 = train_fe[col].value_counts()
    vc2 = test_fe[col].value_counts()
    print(f"train_fe : {col}: {len(vc1)}: {vc1.index[:7]}")
    print(f"test_fe  : {col}: {len(vc2)}: {vc2.index[:7]}")

train_fe : Field_2: 974: Index([           -9999999, 2018-12-28 00:00:00, 2018-12-27 00:00:00,
       2018-12-31 00:00:00, 2019-10-31 00:00:00, 2018-12-26 00:00:00,
       2017-03-24 00:00:00],
      dtype='object')
test_fe  : Field_2: 810: Index([           -9999999, 2018-12-28 00:00:00, 2018-12-27 00:00:00,
       2018-12-31 00:00:00, 2018-12-26 00:00:00, 2019-10-31 00:00:00,
       2019-01-02 00:00:00],
      dtype='object')
train_fe : Field_3: 3: Float64Index([-9999999.0, 2.0, 1.0], dtype='float64')
test_fe  : Field_3: 3: Float64Index([-9999999.0, 2.0, 1.0], dtype='float64')
train_fe : Field_4: 3: CategoricalIndex(['nan', 't1', 'gh'], categories=['gh', 'nan', 't1'], ordered=False, dtype='category')
test_fe  : Field_4: 3: CategoricalIndex(['nan', 't1', 'gh'], categories=['gh', 'nan', 't1'], ordered=False, dtype='category')
train_fe : Field_5: 1902: Index([           -9999999, 2018-12-27 00:00:00, 2018-12-28 00:00:00,
       2018-12-26 00:00:00, 2018-12-25 00:00:00, 2018-12-29 00:00:

train_fe : data.basic_info.locale: 28: CategoricalIndex(['nan', 'vi_vn', 'en_us', 'zh_tw', 'en_gb', 'es_la', 'pt_br'], categories=['_vn', 'af_za', 'ar_ar', 'cb_iq', 'cs_cz', 'cy_gb', 'de_de', 'en_gb', ...], ordered=False, dtype='category')
test_fe  : data.basic_info.locale: 28: CategoricalIndex(['nan', 'vi_vn', 'en_us', 'zh_tw', 'en_gb', 'es_la', 'pt_br'], categories=['_vn', 'af_za', 'ar_ar', 'cb_iq', 'cs_cz', 'cy_gb', 'de_de', 'en_gb', ...], ordered=False, dtype='category')
train_fe : currentLocationCountry: 52: CategoricalIndex(['nan', 'vietnam', 'united states', 'japan', 'south korea',
                  'china', 'taiwan'],
                 categories=['angola', 'argentina', 'australia', 'bangladesh', 'bulgaria', 'cambodia', 'canada', 'china', ...], ordered=False, dtype='category')
test_fe  : currentLocationCountry: 52: CategoricalIndex(['nan', 'vietnam', 'united states', 'japan', 'malaysia',
                  'south korea', 'united kingdom'],
                 categories=['angola', '

train_fe : G_endDate: 231: Index([           -9999999, 2019-10-01 00:00:00, 2019-10-16 00:00:00,
       2019-12-13 00:00:00, 2019-10-04 00:00:00, 2019-09-30 00:00:00,
       2019-10-03 00:00:00],
      dtype='object')
test_fe  : G_endDate: 257: Index([           -9999999, 2020-03-31 00:00:00, 2020-03-30 00:00:00,
       2020-03-23 00:00:00, 2020-03-16 00:00:00, 2020-03-20 00:00:00,
       2020-03-17 00:00:00],
      dtype='object')
train_fe : A_numOrg: 15: Float64Index([-9999999.0, 2.0, 3.0, 1.0, 4.0, 5.0, 6.0], dtype='float64')
test_fe  : A_numOrg: 17: Float64Index([-9999999.0, 2.0, 1.0, 3.0, 4.0, 5.0, 6.0], dtype='float64')
train_fe : A_numQuery: 37: Float64Index([-9999999.0, 2.0, 3.0, 1.0, 4.0, 5.0, 6.0], dtype='float64')
test_fe  : A_numQuery: 39: Float64Index([-9999999.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0], dtype='float64')
train_fe : A_startDate: 400: Index([           -9999999, 2019-09-04 00:00:00, 2019-08-28 00:00:00,
       2019-08-29 00:00:00, 2019-09-03 00:00:00, 2019-08-27 00:00

train_fe : Field_1Field_44: 1925: Float64Index([-9999999.0, 824.0, 267.0, 269.0, 265.0, 273.0, 183.0], dtype='float64')
test_fe  : Field_1Field_44: 1570: Float64Index([-9999999.0, 265.0, 273.0, 182.0, 824.0, 183.0, 269.0], dtype='float64')
train_fe : Field_1Field_7: 1336: Float64Index([-9999999.0, 5.0, 6.0, 4.0, 7.0, 3.0, 8.0], dtype='float64')
test_fe  : Field_1Field_7: 958: Float64Index([-9999999.0, 5.0, 6.0, 4.0, 7.0, 3.0, 0.0], dtype='float64')
train_fe : Field_1Field_8: 1404: Float64Index([-9999999.0, 369.0, 370.0, 368.0, 371.0, 367.0, 372.0], dtype='float64')
test_fe  : Field_1Field_8: 897: Float64Index([-9999999.0, 369.0, 370.0, 368.0, 371.0, 367.0, 372.0], dtype='float64')
train_fe : Field_1Field_9: 1351: Float64Index([-9999999.0, 0.0, 1.0, -7.0, -70.0, -62.0, -28.0], dtype='float64')
test_fe  : Field_1Field_9: 786: Float64Index([-9999999.0, 0.0, 1.0, -7.0, -94.0, -344.0, -62.0], dtype='float64')
train_fe : Field_2Field_43: 1342: Float64Index([-9999999.0, 0.0, 234.0, -706.0, -6

In [57]:
train_fe.Field_10.value_counts()

-9999999.0    27466
 1.0          23354
 2.0           1978
 3.0            201
 4.0             26
 5.0              3
 0.0              2
Name: Field_10, dtype: int64

In [26]:
test_fe.Field_10.value_counts()

-99999.0    11321
 1.0         8224
 2.0          767
 3.0           60
 4.0            8
 5.0            1
Name: Field_10, dtype: int64

In [27]:
data_version = "02-01-2021-v3"

In [28]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
df_train.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test_fe.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)